In [26]:
import numpy as np
from numpy.linalg import norm

In [27]:
from sklearn import datasets, neighbors, linear_model

digits = datasets.load_digits()
X_digits = digits.data
y_digits = digits.target

n_samples = len(X_digits)
n_train = int(.9 * n_samples)

X_train = X_digits[:n_train]
y_train = y_digits[:n_train]
X_test = X_digits[n_train:]
y_test = y_digits[n_train:]

logistic = linear_model.LogisticRegression()

print('LogisticRegression score: %f' % logistic.fit(X_train, y_train).score(X_test, y_test))

LogisticRegression score: 0.938889


In [28]:
def gradient_descent(loss_func, init_theta, alpha=0.5, max_iter=5000, min_diff=1e-4):
    cur_theta = np.float64(init_theta)
    (loss, grad) = loss_func(cur_theta)
    n_iter = 0
    while n_iter < max_iter and abs(loss) > min_diff:
        cur_theta -= alpha * grad
        (loss, grad) = loss_func(cur_theta)
        n_iter += 1
    return cur_theta

In [29]:
# stochastic gradient descent
def sgd(loss_func, init_theta, X, y, alpha=0.5, max_iter=500, n_samples=1000, min_diff=1e-4):
    cur_theta = np.float64(init_theta)
    loss_sum = np.float64(0)
    count = 0
    for _ in range(0, max_iter):
        for i in range(0, len(X)):
            (loss, grad) = loss_func(cur_theta, X[i], y[i])
            loss_sum += loss
            cur_theta -= alpha * grad
            count += 1
            if count == n_samples:
                if abs(loss_sum / n_samples) < min_diff:
                    return cur_theta
                count = 0
                loss_sum = np.float64(0)
    return cur_theta

In [30]:
import numpy as np

class SoftmaxRegression():
    def __init__(self, use_sgd=True):
        self.Theta = None
        self.label2index = None
        self.use_sgd = use_sgd
        
    def fit(self, X_train, y_train):
        (m, n) = X_train.shape
        X = np.c_[ np.ones(m), X_train ]
        y = self.to_vector(y_train)
        K = len(self.label2index)
        init_theta = np.zeros([K, n+1])
        if self.use_sgd:
            self.Theta = sgd(self.loss_function_i, init_theta, X, y)
        else:
            loss_func = lambda theta: self.loss_function(theta, X, y)
            self.Theta = gradient_descent(loss_func, init_theta)
        return self
    
    def predict(self, X_test):
        (m, n) = X_test.shape
        X = np.c_[ np.ones(m), X_test ]
        h = np.empty([m, len(self.label2index)])
        for i in range(0, m):
            powers = self.Theta @ X[i]
            # subtract from the max power to avoid overflow
            exps = np.exp(powers - np.max(powers))
            probs = exps / np.sum(exps)
            h[i][:] = probs
        return h
    
    def score(self, X_test, y_test):
        (m, n) = X_test.shape
        X = np.c_[ np.ones(m), X_test ]
        n_correct = 0
        index2label = { v: k for k, v in self.label2index.items() }
        for i in range(0, m):
            powers = self.Theta @ X[i]
            exps = np.exp(powers - np.max(powers))
            if y_test[i] == index2label[np.argmax(exps)]:
                n_correct += 1
        return n_correct / m
    
    # convert label to a column of identity matrix I_K
    def to_vector(self, y, new_label=True):
        if new_label:
            y_list = np.unique(y).tolist()
            self.label2index = { y_i: y_list.index(y_i) for y_i in y_list }
        n_labels = len(y_list)
        return np.array([ np.eye(n_labels)[self.label2index[y_i]] for y_i in y ])

    def loss_function_i(self, Theta, x_i, y_i):
        K = Theta.shape[0]
        n = len(x_i)
        powers = Theta @ x_i
        # subtract from the max power to avoid overflow
        exps = np.exp(powers - np.max(powers))
        # take maximum of probs and a tiny value to avoid taking log of 0
        probs = np.maximum(exps / np.sum(exps), np.finfo(float).tiny)
        cost_i = -1 * y_i @ np.log(probs)
        grad_i = -1 * (y_i - probs).reshape([K, 1]) @ x_i.reshape([1, n])
        return (cost_i, grad_i)
    
    def loss_function(self, Theta, X, y):
        m = X.shape[0]
        cost = np.float64(0)
        grad = np.zeros(Theta.shape)
        for i in range(0, m):
            (cost_i, grad_i) = self.loss_function_i(Theta, X[i], y[i])
            cost += cost_i
            grad += grad_i
        return (cost / m, grad / m)

In [31]:
softmax = SoftmaxRegression()
softmax.fit(X_train, y_train)

In [32]:
softmax.score(X_test, y_test)

0.9277777777777778